# Tool calling

[How to create tool](https://python.langchain.com/v0.2/docs/how_to/custom_tools/#structuredtool)

In [9]:
from langchain_openai import ChatOpenAI
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model="gpt-4o")

## Create tool with `@tool`

In [5]:
class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


@tool("multiplication-tool", args_schema=CalculatorInput, return_direct=True)
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.return_direct)

multiplication-tool
Multiply two numbers.
{'a': {'title': 'A', 'description': 'first number', 'type': 'integer'}, 'b': {'title': 'B', 'description': 'second number', 'type': 'integer'}}
True


In [6]:
tools = [multiply]
tools

[StructuredTool(name='multiplication-tool', description='Multiply two numbers.', args_schema=<class '__main__.CalculatorInput'>, return_direct=True, func=<function multiply at 0x112901510>)]

## Agent 

In [7]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [10]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='d0cdc4b9-1b18-47ce-ab64-9028e5bf39b3'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 54, 'total_tokens': 64}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_4e2b2da518', 'finish_reason': 'stop', 'logprobs': None}, id='run-392c8f25-5c93-4556-9a36-3fff913193bd-0')]

In [11]:
response = agent_executor.invoke({"messages": [HumanMessage(content="What is the result of one times two?")]})

response["messages"]

[HumanMessage(content='What is the result of one times two?', id='05e17659-2b55-4e96-a2e0-ec8bdac0a0dd'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_KUlWCagj3gpN41Sccu9fvWZ6', 'function': {'arguments': '{"a":1,"b":2}', 'name': 'multiplication-tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 61, 'total_tokens': 81}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_3cd8b62c3b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c6d7b361-d22e-457f-b88c-1bf015b569f5-0', tool_calls=[{'name': 'multiplication-tool', 'args': {'a': 1, 'b': 2}, 'id': 'call_KUlWCagj3gpN41Sccu9fvWZ6', 'type': 'tool_call'}]),
 ToolMessage(content='2', name='multiplication-tool', id='b4ebe5b9-8787-423d-9ef1-91c4632bf882', tool_call_id='call_KUlWCagj3gpN41Sccu9fvWZ6'),
 AIMessage(content='The result of one times two is two.', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 91, 'total_tokens': 101

In [14]:
response["messages"][1].additional_kwargs

{'tool_calls': [{'id': 'call_KUlWCagj3gpN41Sccu9fvWZ6',
   'function': {'arguments': '{"a":1,"b":2}', 'name': 'multiplication-tool'},
   'type': 'function'}]}